In [1]:
import numpy as np
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, interplevel, ll_to_xy)
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import xarray as xr
import gc, glob
import pickle
from tqdm import tqdm

In [2]:
import sys
sys.path.insert(1, '../../')
from util.wrf_process import (calc_derive, object_tracking, read_and_write, regrid)
import read_config

# Configuration

In [3]:
# Read configuration file
config_set = read_config.read_config('../../config.ini')
# Experiment Type
if config_set['track_exp']=='CTRL':
    i_senstest = False
else:
    i_senstest = True
    
# Open the NetCDF file
ncfile = Dataset('../../storage/memb01/wrfout_d02_2013-11-02_00:00:00')
# Get basic domain settings
lons,lats,pres = read_and_write.get_basic_domain(ncfile)

if (lons.min() < 0) and (lons.max() > 0):
    lon_offset = object_tracking.dateline_lon_shift(lons, reverse=0)
else:
    lon_offset = 0

basis=0

# Read files

In [ ]:
for memb in [10]:#[1,2,3,4,5,6,7,8,9]:
    relvorts = np.asarray([read_and_write.depickle(obj) for obj in sorted(glob.glob(f'../../storage/memb{memb}/proc/relvort/*'))])
    
    # Run tracking
    track, f_masked = object_tracking.object_track(relvorts, 
                                                   (lons + lon_offset).data, 
                                                   (lats).data, 
                                                   i_senstest, 
                                                   basis, 
                                                   int(config_set['track_nxsm']),
                                                   int(config_set['track_nxrepeat']),
                                                   int(config_set['track_ntsmooth']),
                                                   int(config_set['track_rmax']),
                                                  )
    
    track_xy = object_tracking.tree_latlon_to_xy(ncfile,
                                                 (lons + lon_offset).data,
                                                 (lats).data,
                                                 track.data[0][1:],
                                                 track.data[1][1:],
                                                )
    trackdict = {'trackobj':track_xy, 'data':f_masked}
    filepath = f"../../storage/memb{memb}/proc/track.pkl"
    read_and_write.save_to_pickle(trackdict,filepath)

# Cut data

In [4]:
def do_vert_pol(trackobj=None,varobj=None,radius=175):
    saved = []
    for ivert in range(varobj.shape[0]):
        tmp = varobj[ivert,trackobj[1]-radius:trackobj[1]+radius,trackobj[0]-radius:trackobj[0]+radius]
        tmp_pol,r_grid,theta_grid = regrid.cart_image_to_pol(tmp,Jacobian=False,dr=1,dt=np.pi/180)
        saved.append(tmp_pol)
        del tmp,tmp_pol
        gc.collect()
    return saved,r_grid,theta_grid

def do_vert_cart(trackobj=None,varobj=None,radius=175):
    saved = []
    for ivert in range(varobj.shape[0]):
        tmp = varobj[ivert,trackobj[1]-radius:trackobj[1]+radius,trackobj[0]-radius:trackobj[0]+radius]
        #tmp_pol,r_grid,theta_grid = regrid.cart_image_to_pol(tmp,Jacobian=False,dr=1,dt=np.pi/180)
        saved.append(tmp)
        del tmp
        gc.collect()
    return saved

In [22]:
for memb in [10]:#[1,2,3,4,5,6,7,8,9]:
    rthratlw = [read_and_write.depickle(obj) for obj in sorted(glob.glob(f'../../storage/memb{memb}/proc/V/*'))]
    track = read_and_write.depickle(f'../../storage/memb{memb}/proc/track.pkl')
    
    savepol,saver,savetheta,saveitmp=[],[],[],[]
    for itmp in tqdm(range(len(rthratlw))):
        try:
            tmp,r_grid,theta_grid = do_vert_pol(trackobj=track['trackobj']['latlon_xy'][itmp],varobj=rthratlw[itmp],radius=175)
            if ((np.asarray(tmp).shape[1]==248) and (np.asarray(tmp).shape[2]!=0)):
                savepol.append(np.asarray(tmp))
                saver.append(np.asarray(r_grid))
                savetheta.append(np.asarray(theta_grid))
                saveitmp.append(itmp)
                del tmp
                gc.collect()
            else:
                continue
        except:
            continue

    filepath = f"../../storage/memb{memb}/proc/cutpol_V.pkl"
    read_and_write.save_to_pickle({'pol':savepol,'itmp':saveitmp,'r':saver,'theta':savetheta},filepath)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [02:22<00:00,  1.98s/it]


In [6]:
for memb in [1,2,3,4,5,6,7,8,9]:
    rthratlw = [read_and_write.depickle(obj) for obj in sorted(glob.glob(f'../../storage/memb0{memb}/proc/RTHRATSW/*'))]
    track = read_and_write.depickle(f'../../storage/memb0{memb}/proc/track.pkl')
    
    savepol,saveitmp=[],[]
    for itmp in tqdm(range(len(rthratlw))):
        try:
            tmp = do_vert_cart(trackobj=track['trackobj']['latlon_xy'][itmp],varobj=rthratlw[itmp],radius=175)
            if ((np.asarray(tmp).shape[1]==350) and (np.asarray(tmp).shape[2]==350)):
                savepol.append(np.asarray(tmp))
                saveitmp.append(itmp)
                del tmp
                gc.collect()
            else:
                continue
        except:
            continue

    filepath = f"../../storage/memb0{memb}/proc/cutcart_rthratsw.pkl"
    read_and_write.save_to_pickle({'pol':savepol,'itmp':saveitmp},filepath)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [03:23<00:00,  2.83s/it]
